In [ ]:
!pip install ffmpeg-python tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 9.4 MB/s eta 0:00:00


In [ ]:
# Вывод информации о входном видео
!ffprobe -v error -show_streams /content/drive/MyDrive/CV_SBER/curling_video.mp4

[STREAM]
index=0
codec_name=h264
codec_long_name=H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10
profile=High
codec_type=video
codec_tag_string=avc1
codec_tag=0x31637661
width=1920
height=1080
coded_width=1920
coded_height=1080
closed_captions=0
has_b_frames=1
sample_aspect_ratio=1:1
display_aspect_ratio=16:9
pix_fmt=yuv420p
level=40
color_range=tv
color_space=bt709
color_transfer=bt709
color_primaries=bt709
chroma_location=left
field_order=unknown
refs=1
is_avc=true
nal_length_size=4
id=N/A
r_frame_rate=30/1
avg_frame_rate=706093349/23536480
time_base=1/90000
start_pts=5999
start_time=0.066656
duration_ts=1114567659
duration=12384.085100
bit_rate=3647432
max_bit_rate=N/A
bits_per_raw_sample=8
nb_frames=371522
nb_read_frames=N/A
nb_read_packets=N/A
DISPOSITION:default=1
DISPOSITION:dub=0
DISPOSITION:original=0
DISPOSITION:comment=0
DISPOSITION:lyrics=0
DISPOSITION:karaoke=0
DISPOSITION:forced=0
DISPOSITION:hearing_impaired=0
DISPOSITION:visual_impaired=0
DISPOSITION:clean_effects=0
DISPOSITI

In [ ]:
!pip install uv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 68.6 MB/s eta 0:00:00


In [ ]:
!uv pip install opencv-python torch torchvision scikit-image -q

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import os
import math

In [ ]:
import subprocess
import re
from google.colab import files
from tqdm import tqdm
import os

# Подключаем Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Параметры
input_video = "/content/drive/MyDrive/CV_SBER/curling_video.mp4"  # Имя входного видеофайла
start_time = 1880  # Начало вырезки (в секундах)
end_time = 5480    # ТЕСТ: 10 секунд для проверки
output_resolution = (1094, 540)  # Новое разрешение (ширина, высота)
temp_video = "/content/drive/MyDrive/CV_SBER/temp_cut_video.mp4"  # Временный файл на Google Drive
final_output = "/content/drive/MyDrive/CV_SBER/Участок_с_полного_матча.mp4"  # Финальный файл на Google Drive

# Проверка существования файла
if not os.path.exists(input_video):
    print(f"Ошибка: файл {input_video} не найден")
    exit()

# Длительность вырезаемого участка (в секундах)
duration = end_time - start_time

# Этап 1: Вырезаем участок с копированием потоков
cut_command = [
    "ffmpeg", "-i", input_video,
    "-ss", str(start_time), "-to", str(end_time),
    "-c:v", "copy", "-c:a", "copy", temp_video, "-y",
    "-loglevel", "error"  # Минимизируем вывод
]
print("Вырезаем участок видео...")
result = subprocess.run(cut_command, capture_output=True, text=True)
if result.returncode != 0:
    print(f"Ошибка вырезки: {result.stderr}")
    exit()

# Проверка, создан ли временный файл
if not os.path.exists(temp_video):
    print(f"Ошибка: временный файл {temp_video} не создан")
    exit()

# Этап 2: Обработка (crop, scale, pad)
ffmpeg_command = [
    "ffmpeg", "-i", temp_video,
    "-vf", f"crop=1920:980:0:0,scale=1094:540:force_original_aspect_ratio=decrease,pad=1094:540:(ow-iw)/2:(oh-ih)/2",
    "-pix_fmt", "yuv420p",  # Формат пикселей
    "-c:v", "libx264", "-profile:v", "main", "-preset", "fast",
    "-c:a", "copy",  # Копируем аудио
    "-threads", "0", final_output, "-y",
    "-loglevel", "verbose"
]

# Запускаем ffmpeg с прогресс-баром
process = subprocess.Popen(ffmpeg_command, stderr=subprocess.PIPE, universal_newlines=True)
pbar = tqdm(total=duration, desc="Processing video", unit="s")
time_pattern = re.compile(r"time=(\d+):(\d+):(\d+\.\d+)")

for line in process.stderr:
    print(line.strip())  # Выводим логи для отладки
    match = time_pattern.search(line)
    if match:
        hours, minutes, seconds = map(float, match.groups())
        current_time = hours * 3600 + minutes * 60 + seconds
        pbar.n = min(current_time, duration)
        pbar.refresh()

pbar.close()
process.wait()

# Проверка результата
if process.returncode == 0:
    print("Обработка завершена успешно!")
    # Проверка разрешения
    import cv2
    cap = cv2.VideoCapture(final_output)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Выходное разрешение: {width}x{height}")
    cap.release()
    files.download(final_output)
else:
    print("Ошибка при выполнении ffmpeg")

# Очистка временных файлов
if os.path.exists(temp_video):
    os.remove(temp_video)

# Проверка ресурсов
print("\nПроверка ресурсов:")
!df -h /content/drive
!free -h

Mounted at /content/drive
Вырезаем участок видео...


Processing video:   0%|          | 0/3600 [00:00<?, ?s/s]

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

Processing video:   0%|          | 1.57/3600 [00:00<35:08,  1.71s/s]

frame=   35 fps=0.0 q=0.0 size=       0kB time=00:00:01.57 bitrate=   0.2kbits/s dup=34 drop=0 speed=32.5x


...

Processing video: 100%|█████████▉| 3599.97/3600 [16:30<00:00,  3.63s/s]


frame=108002 fps=109 q=-1.0 Lsize=  459802kB time=00:59:59.97 bitrate=1046.3kbits/s dup=34 drop=0 speed=3.64x
video:339442kB audio:115200kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.135091%
Input file #0 (/content/drive/MyDrive/CV_SBER/temp_cut_video.mp4):
Input stream #0:0 (video): 107968 packets read (2245650728 bytes); 107968 frames decoded;
Input stream #0:1 (audio): 168750 packets read (117964800 bytes);
Total: 276718 packets (2363615528 bytes) demuxed
Output file #0 (/content/drive/MyDrive/CV_SBER/Участок_с_полного_матча.mp4):
Output stream #0:0 (video): 108002 frames encoded; 108002 packets muxed (347588439 bytes);
Output stream #0:1 (audio): 168750 packets muxed (117964800 bytes);
Total: 276752 packets (465553239 bytes) muxed
[AVIOContext @ 0x59bb38781580] Statistics: 32 seeks, 1817 writeouts
[libx264 @ 0x59bb38633d40] frame I:519   Avg QP:19.29  size: 50382
[libx264 @ 0x59bb38633d40] frame P:32548 Avg QP:21.90  size:  6620
[libx264 @ 0x59bb38633d40] 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Проверка ресурсов:
Filesystem      Size  Used Avail Use% Mounted on
drive            15G   12G  3.8G  75% /content/drive
               total        used        free      shared  buff/cache   available
Mem:           334Gi       2.5Gi       302Gi       4.0Mi        29Gi       329Gi
Swap:             0B          0B          0B


In [ ]:
# Вывод информации о видео
!ffprobe -v error -show_streams /content/drive/MyDrive/CV_SBER/Участок_с_полного_матча.mp4

[STREAM]
index=0
codec_name=h264
codec_long_name=H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10
profile=Main
codec_type=video
codec_tag_string=avc1
codec_tag=0x31637661
width=1094
height=540
coded_width=1094
coded_height=540
closed_captions=0
has_b_frames=2
sample_aspect_ratio=25920:25921
display_aspect_ratio=52512:25921
pix_fmt=yuv420p
level=31
color_range=tv
color_space=bt709
color_transfer=bt709
color_primaries=bt709
chroma_location=left
field_order=unknown
refs=1
is_avc=true
nal_length_size=4
id=N/A
r_frame_rate=30/1
avg_frame_rate=30/1
time_base=1/15360
start_pts=0
start_time=0.000000
duration_ts=55297024
duration=3600.066667
bit_rate=772404
max_bit_rate=N/A
bits_per_raw_sample=8
nb_frames=108002
nb_read_frames=N/A
nb_read_packets=N/A
DISPOSITION:default=1
DISPOSITION:dub=0
DISPOSITION:original=0
DISPOSITION:comment=0
DISPOSITION:lyrics=0
DISPOSITION:karaoke=0
DISPOSITION:forced=0
DISPOSITION:hearing_impaired=0
DISPOSITION:visual_impaired=0
DISPOSITION:clean_effects=0
DISPOSITION:attac

In [ ]:
# Функция вычисления разницы между кадрами
def calculate_frame_difference(frame1, frame2):
    return np.mean((frame1 - frame2) ** 2)

# Выбор представительного кадра из сегмента
def get_segment_representative(frames):
    return frames[len(frames) // 2]

# Функция для вычисления MSE
def calculate_mse(img1, img2):
    return np.mean((img1 - img2) ** 2)

# Функция классификации по метрикам SSIM и MSE
def classify_segment(rep_frame, ref_frames, threshold):

    """

    rep_frame - кадр, проходящий отбор (классификацию)
    ref_frames - 2 эталонных кадра (должно быть 2 пути)
    threshold - пороговое знаяение метрики для классификации

    """

    # Конвертируем тестовый кадр в градации серого
    rep_frame_gray = cv2.cvtColor(rep_frame, cv2.COLOR_BGR2GRAY)


    similarities = []
    mse_values = []

    for i, ref in enumerate(ref_frames):
        # Конвертируем эталонный кадр в градации серого
        ref_gray = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)

        # Вычисляем SSIM
        ssim_score = ssim(rep_frame_gray, ref_gray)
        similarities.append(ssim_score)

        # Вычисляем MSE
        mse_score = calculate_mse(rep_frame_gray, ref_gray)
        mse_values.append(mse_score)


    # Определяем максимальное сходство по SSIM
    max_similarity = max(similarities)

    # Классификация (на основе SSIM, как в исходной функции)
    if max_similarity > threshold:
        angle_index = np.argmax(similarities)
        return angle_index
    elif max_similarity > 0.5 and mse_score < 80:
        angle_index = np.argmax(similarities)
        return angle_index
    else:
        return -1

# Основная функция отбора кадров
def select_frames(video_path, ref_frame_paths, output_dir, diff_threshold=40, ssim_threshold=0.6, frame_skip=1):

    """
    video_path - путь к видео
    ref_frame_paths - 2 эталонных кадра (должно быть 2 пути)
    output_dir - папка сохранения отобранных кадров
    diff_threshold - размер сегмента, проходящего отбор (при diff_threshold=50 из видео берётся 50 последовательных кадров
                     и один из кадров проходит процесс отбор и в случае успеха все 50 кадров сохраняются в папке для отобранных кадров)
    ssim_threshold - пороговое знаяение метрики для классификации
    frame_skip - через сколько кадров вытаскивать (при frame_skip=1 для отбора будет вытаскиваться каждый кадр)

    """

    # Загрузка эталонных кадров
    ref_frames = [cv2.imread(path) for path in ref_frame_paths]
    if any(ref is None for ref in ref_frames):
        raise ValueError("Ошибка загрузки эталонных кадров")

    # Создание папки для сохранения кадров
    os.makedirs(output_dir, exist_ok=True)

    # Открытие видео
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Ошибка открытия видео")

    prev_frame = None
    segments = []
    current_segment = []
    frame_count = 0

    # Разделение видео на сегменты
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Пропуск кадров
        if frame_count % frame_skip != 0:
            frame_count += 1
            continue

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            diff = calculate_frame_difference(gray_frame, prev_frame)
            if diff > diff_threshold:
                if current_segment:
                    segments.append(current_segment)
                current_segment = []
            current_segment.append((frame_count, frame))

        prev_frame = gray_frame
        frame_count += 1

    if current_segment:
        segments.append(current_segment)

    # Классификация и сохранение кадров
    selected_frames = []
    for segment in segments:
        rep_frame = get_segment_representative([frame for _, frame in segment])
        angle_index = classify_segment(rep_frame, ref_frames, ssim_threshold)
        if angle_index != -1:
            for frame_num, frame in segment:
                output_path = os.path.join(output_dir, f"angle_{angle_index}_frame_{frame_num}.jpg")
                cv2.imwrite(output_path, frame)
                selected_frames.append((frame_num, frame, angle_index))

    cap.release()
    return selected_frames

In [ ]:
# Параметры
ref_frame1 = "/content/drive/MyDrive/CV_SBER/Эталонные_углы_обзора/Эталон_1.jpg"
ref_frame2 = "/content/drive/MyDrive/CV_SBER/Эталонные_углы_обзора/Эталон_2.jpg"
video_path = "/content/drive/MyDrive/CV_SBER/Участок_с_полного_матча.mp4"
output_dir = "/content/drive/MyDrive/CV_SBER/Отобранные_по_2_ум_углам_кадры/"

# Выполнение отбора кадров
try:
    selected_frames = select_frames(video_path, [ref_frame1, ref_frame2], output_dir)
    print(f"Отобрано {len(selected_frames)} кадров")
    angle_1 = 0
    angle_2 = 0
    for frame_num, _, angle_index in selected_frames:
        if angle_index + 1 == 1:
          angle_1 += 1
        elif angle_index + 1 == 2:
          angle_2 += 1
    print(f"Для 1-ого угла обзора: {angle_1}")
    print(f"Для 2-ого угла обзора: {angle_2}")
except Exception as e:
    print(f"Ошибка: {e}")


Отобрано 42236 кадров
Для 1-ого угла обзора: 1514
Для 2-ого угла обзора: 40722


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
import glob
import gc
import torch

МАСКА ИСКЛЮЧЕНИЯ ИЗ ВНИМАНИЯ БАННЕРОВ

In [ ]:
def create_text_mask_by_location(image):
    h, w = image.shape[:2]
    # Создаем пустую маску (255 - область интереса, 0 - текст/баннеры)
    text_mask = np.ones((h, w), dtype=np.uint8) * 255

    # Координаты баннеров (исправленные на основе предположений)
    # Первый баннер: [45, 474] (x1, y1) и [334, 524] (x2, y2)
    x1, y1 = min(45, 334), min(474, 524)  # Верхний левый угол
    x2, y2 = max(45, 334), max(474, 524)  # Нижний правый угол
    text_mask[y1:y2, x1:x2] = 0

    # Второй баннер: [879, 2] (x1, y1) и [1072, 50] (x2, y2)
    x1, y1 = min(879, 1072), min(2, 50)  # Верхний левый угол
    x2, y2 = max(879, 1072), max(2, 50)  # Нижний правый угол
    text_mask[y1:y2, x1:x2] = 0


    # Преобразование исходного изображения в HSV для создания маски фона
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Диапазоны для синего фона (настройте под ваш кадр)
    lower_blue = np.array([100, 200, 150])  # Уточненный диапазон для синего
    upper_blue = np.array([130, 255, 255])
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # Инвертируем маску фона (0 - синий фон, 255 - остальное)
    mask_blue = cv2.bitwise_not(mask_blue)

    # Объединяем маску баннеров и маску фона
    final_mask = cv2.bitwise_and(text_mask, mask_blue)

    return final_mask



МАСКА ДЛЯ "ДОМА"

In [ ]:
# Новая функция для создания маски "дома"
def create_house_mask(image, text_mask):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Диапазон для "дома"
    lower_house = np.array([105, 25, 75])
    upper_house = np.array([125, 178, 130])

    mask_house = cv2.inRange(hsv, lower_house, upper_house)

    # Применяем маску баннеров, чтобы исключить текст
    mask_house = cv2.bitwise_and(mask_house, mask_house, mask=text_mask)

    # Морфологические операции для улучшения маски
    kernel = np.ones((5, 5), np.uint8)
    mask_house = cv2.morphologyEx(mask_house, cv2.MORPH_OPEN, kernel)  # Удаление шума
    mask_house = cv2.morphologyEx(mask_house, cv2.MORPH_CLOSE, kernel)  # Заполнение разрывов


    # Поиск контуров "дома"
    contours_house, _ = cv2.findContours(mask_house, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Фильтрация контуров по площади
    filtered_contours_house = [cnt for cnt in contours_house if cv2.contourArea(cnt) > 16000]

    return mask_house, filtered_contours_house

МАСКА ДЛЯ КАМНЕЙ (ЖЁЛТЫХ И КРАСНЫХ)

In [ ]:
def find_contours_of_cards(image, text_mask):
    # Применяем маску текста, чтобы исключить баннеры
    image_masked = cv2.bitwise_and(image, image, mask=text_mask)

    # Преобразование в HSV для камней
    hsv = cv2.cvtColor(image_masked, cv2.COLOR_BGR2HSV)

    # Диапазоны для желтых камней
    lower_yellow = np.array([20, 50, 130])
    upper_yellow = np.array([30, 255, 255])
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Диапазоны для красных камней (два диапазона, так как красный цвет в HSV делится на два участка)
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 230, 230])
    lower_red2 = np.array([160, 50, 50])
    upper_red2 = np.array([180, 230, 230])
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)

    # Диапазон для серого (для учета серой части камней)
    lower_gray = np.array([0, 0, 60])
    upper_gray = np.array([50, 100, 180])
    mask_gray = cv2.inRange(hsv, lower_gray, upper_gray)

    # Объединяем маски для желтых и серых частей камней
    mask_yellow_combined = cv2.bitwise_or(mask_yellow, mask_gray)
    # Объединяем маски для красных и серых частей камней
    mask_red_combined = cv2.bitwise_or(mask_red, mask_gray)

    ## Применяем маску текста к каждой цветной маске
    mask_yellow_combined = cv2.bitwise_and(mask_yellow_combined, mask_yellow_combined, mask=text_mask)
    mask_red_combined = cv2.bitwise_and(mask_red_combined, mask_red_combined, mask=text_mask)

    # Морфологические операции для очистки масок
    kernel = np.ones((5, 5), np.uint8)
    mask_yellow_combined = cv2.morphologyEx(mask_yellow_combined, cv2.MORPH_OPEN, kernel)
    mask_yellow_combined = cv2.morphologyEx(mask_yellow_combined, cv2.MORPH_CLOSE, kernel)
    mask_red_combined = cv2.morphologyEx(mask_red_combined, cv2.MORPH_OPEN, kernel)
    mask_red_combined = cv2.morphologyEx(mask_red_combined, cv2.MORPH_CLOSE, kernel)

    # Поиск контуров для желтых камней
    contours_yellow, _ = cv2.findContours(mask_yellow_combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours_yellow = [cnt for cnt in contours_yellow if 400 < cv2.contourArea(cnt) < 1500]

    # Поиск контуров для красных камней
    contours_red, _ = cv2.findContours(mask_red_combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours_red = [cnt for cnt in contours_red if 400 < cv2.contourArea(cnt) < 1500]

    def filter_ellipses(contours, mask_color, mask_gray):
        confirmed_contours = []
        for cnt in contours:
            # Проверяем, достаточно ли точек для аппроксимации эллипса (не менее 5)
            if len(cnt) >= 5:
                x, y, w, h = cv2.boundingRect(cnt)
                # Проверяем, что ROI не пустой
                if w > 0 and h > 0:
                    try:
                        # Аппроксимация эллипса
                        ellipse = cv2.fitEllipse(cnt)
                        (center_x, center_y), (major_axis, minor_axis), angle = ellipse


                        # Проверяем, соответствует ли эллипс размерам контура
                        axis_ratio = min(major_axis, minor_axis) / max(major_axis, minor_axis)

                        if 0.7 < axis_ratio < 1.3:  # Допустимое соотношение осей
                            if abs(major_axis - w) < 15 and abs(minor_axis - h) < 15:
                                # Проверяем наличие цветной части (желтой или красной) в ROI
                                roi_color = mask_color[y:y+h, x:x+w]
                                roi_gray = mask_gray[y:y+h, x:x+w]
                                color_pixels = cv2.countNonZero(roi_color)
                                gray_pixels = cv2.countNonZero(roi_gray)
                                total_pixels = roi_color.size
                                if total_pixels > 0:
                                    color_ratio = color_pixels / total_pixels
                                    gray_ratio = gray_pixels / total_pixels
                                    # Убедимся, что есть цветная часть (желтая/красная) и серая
                                    if (color_ratio > 0.1 and gray_ratio > 0.1) or (color_ratio > 0.1 and gray_ratio > 0.01):
                                        confirmed_contours.append(cnt)
                    except cv2.error:
                        # Пропускаем контуры, для которых не удалось аппроксимировать эллипс
                        continue
        return confirmed_contours

    # Фильтруем контуры с помощью эллипсов
    confirmed_yellow = filter_ellipses(filtered_contours_yellow, mask_yellow, mask_gray)
    confirmed_red = filter_ellipses(filtered_contours_red, mask_red, mask_gray)

    return confirmed_yellow, confirmed_red, mask_yellow_combined, mask_red_combined

СОЗДАНИЕ АННОТАЦИИ

In [ ]:
# Функция создания аннотации
def create_yolo_annotations(contours_yellow, contours_red, contours_house, frame_number, image_shape, output_dir):
    img_height, img_width = image_shape[:2]
    annotations_yolo = []
    class_map = {"yellow": 0, "red": 1, "house": 2}  # Добавляем класс для "дома"

    for cnt in contours_yellow:
        x, y, w, h = cv2.boundingRect(cnt)
        center_x = x + w // 2
        center_y = y + h // 2
        norm_center_x = center_x / img_width
        norm_center_y = center_y / img_height
        norm_width = w / img_width
        norm_height = h / img_height
        annotations_yolo.append(f"{class_map['yellow']} {norm_center_x:.6f} {norm_center_y:.6f} {norm_width:.6f} {norm_height:.6f}")

    for cnt in contours_red:
        x, y, w, h = cv2.boundingRect(cnt)
        center_x = x + w // 2
        center_y = y + h // 2
        norm_center_x = center_x / img_width
        norm_center_y = center_y / img_height
        norm_width = w / img_width
        norm_height = h / img_height
        annotations_yolo.append(f"{class_map['red']} {norm_center_x:.6f} {norm_center_y:.6f} {norm_width:.6f} {norm_height:.6f}")


    for cnt in contours_house:
        x, y, w, h = cv2.boundingRect(cnt)
        center_x = x + w // 2
        center_y = y + h // 2
        norm_center_x = center_x / img_width
        norm_center_y = center_y / img_height
        norm_width = w / img_width
        norm_height = h / img_height
        annotations_yolo.append(f"{class_map['house']} {norm_center_x:.6f} {norm_center_y:.6f} {norm_width:.6f} {norm_height:.6f}")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    txt_output_path = os.path.join(output_dir, f"frame_{frame_number}.txt")
    with open(txt_output_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(annotations_yolo))


In [ ]:
# Основная функция создания аннотаций к отобранными кадрам
def create_annotation(selected_frames_dir_path, selected_frames_output_dir, output_annotation_dir):

    number_of_selected_frames = 0
    number_of_created_annotations = 0

    # Получаем список всех .jpg файлов в папке
    frame_paths = glob.glob(os.path.join(selected_frames_dir_path, "*.jpg"))
    # Сортировка по имени файла (опционально, для порядка)
    frame_paths.sort()

      # Загрузка кадров
    selected_frames = [cv2.imread(path) for path in frame_paths]
    if any(ref is None for ref in selected_frames):
        raise ValueError("Ошибка загрузки отобранных кадров")

    # Создание папки для сохранения аннотаций
    os.makedirs(output_annotation_dir, exist_ok=True)

    # Создание папки для сохранения кадров
    os.makedirs(selected_frames_output_dir, exist_ok=True)

    # Создание аннотаций
    for selected_frame, frame_path in zip(selected_frames, frame_paths):

        frame_number = ''

        # Создание масок для кадра
        text_mask = create_text_mask_by_location(selected_frame)
        mask_house, contours_house = create_house_mask(selected_frame, text_mask)

        contours_yellow, contours_red, mask_yellow, mask_red = find_contours_of_cards(selected_frame, text_mask)

        # Проверка на пустую аннотацию
        is_annotation_empty = (len(contours_yellow) == 0 and len(contours_red) == 0 and len(contours_house) == 0)

        if is_annotation_empty:
            continue

        # Номер кадра
        frame_filename = os.path.basename(frame_path)  # Получаем имя файла из пути

        if "angle_0_frame_" in frame_filename:
            frame_number = frame_filename.replace('angle_0_frame_', '').replace('.jpg', '')

        elif "angle_1_frame_" in frame_filename:
            frame_number = frame_filename.replace('angle_1_frame_', '').replace('.jpg', '')


        # Создание и сохранение YOLO аннотации (сохранение включено в create_yolo_annotations)
        create_yolo_annotations(contours_yellow, contours_red, contours_house, frame_number, selected_frame.shape, output_dir=output_annotation_dir)
        number_of_created_annotations += 1

        # Сохранение кадра
        output_path = os.path.join(selected_frames_output_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_path, selected_frame)
        number_of_selected_frames += 1

    print(f"Кадры сохранены в: {selected_frames_output_dir}\n Сохранено кадров: {number_of_selected_frames}")
    print(f"Аннотации сохранены в: {output_annotation_dir}\n Сохранено аннотаций: {number_of_created_annotations}")




In [ ]:
# Запуск автоматического создания аннотаций к отобранными кадрам
selected_frames_dir_path = "/content/drive/MyDrive/CV_SBER/Отобранные_по_2_ум_углам_кадры/"
selected_frames_output_dir = "/content/drive/MyDrive/CV_SBER/1_Полный_матч_images"
output_annotation_dir = "/content/drive/MyDrive/CV_SBER/1_Полный_матч_labels"


create_annotation(selected_frames_dir_path, selected_frames_output_dir, output_annotation_dir)

Кадры сохранены в: /content/drive/MyDrive/CV_SBER/1_Полный_матч_images
 Сохранено кадров: 38398
Аннотации сохранены в: /content/drive/MyDrive/CV_SBER/1_Полный_матч_labels
 Сохранено аннотаций: 38398


In [ ]:
!pip install tensorflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

In [ ]:
# Папки с исходными данными
images_dir = "/content/drive/MyDrive/CV_SBER/1_Полный_матч_images"
labels_dir = "/content/drive/MyDrive/CV_SBER/1_Полный_матч_labels"

# Целевая папка для датасета
dataset_dir = "/content/drive/MyDrive/CV_SBER/dataset"
images_train_dir = os.path.join(dataset_dir, "images", "train")
images_val_dir = os.path.join(dataset_dir, "images", "val")
images_test_dir = os.path.join(dataset_dir, "images", "test")
labels_train_dir = os.path.join(dataset_dir, "labels", "train")
labels_val_dir = os.path.join(dataset_dir, "labels", "val")
labels_test_dir = os.path.join(dataset_dir, "labels", "test")

# Создание директорий
os.makedirs(images_train_dir, exist_ok=True)
os.makedirs(images_val_dir, exist_ok=True)
os.makedirs(images_test_dir, exist_ok=True)
os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_val_dir, exist_ok=True)
os.makedirs(labels_test_dir, exist_ok=True)

# Получение списка изображений
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
image_files.sort()  # Сортировка для соответствия с аннотациями

# Проверка наличия аннотаций для каждого изображения
valid_images = []
for img in image_files:
    label_file = os.path.splitext(img)[0] + '.txt'
    if os.path.exists(os.path.join(labels_dir, label_file)):
        valid_images.append(img)
    else:
        print(f"Аннотация для {img} не найдена, пропускается.")

# Берем только первые 6000 пар
valid_images = valid_images[:6000]
print(f"Выбрано {len(valid_images)} изображений с аннотациями")

# Разделение на train, val, test
train_images, temp_images = train_test_split(valid_images, train_size=0.70, random_state=42)
val_images, test_images = train_test_split(temp_images, train_size=0.50, random_state=42)  # 50% от оставшихся 30% = 15%

# Функция для копирования файлов
def copy_files(file_list, src_img_dir, src_label_dir, dst_img_dir, dst_label_dir):
    for img in file_list:
        label = os.path.splitext(img)[0] + '.txt'
        # Копирование изображения
        shutil.copy(os.path.join(src_img_dir, img), os.path.join(dst_img_dir, img))
        # Копирование аннотации
        shutil.copy(os.path.join(src_label_dir, label), os.path.join(dst_label_dir, label))

# Копирование файлов в соответствующие папки
copy_files(train_images, images_dir, labels_dir, images_train_dir, labels_train_dir)
copy_files(val_images, images_dir, labels_dir, images_val_dir, labels_val_dir)
copy_files(test_images, images_dir, labels_dir, images_test_dir, labels_test_dir)

# Проверка количества файлов
print(f"Train: {len(train_images)} изображений")
print(f"Val: {len(val_images)} изображений")
print(f"Test: {len(test_images)} изображений")

# Создание data.yaml
data_yaml = f"""
train: {images_train_dir}
val: {images_val_dir}
test: {images_test_dir}
nc: 3
names: ['yellow', 'red', 'house']
"""

with open(os.path.join(dataset_dir, "data.yaml"), 'w', encoding='utf-8') as f:
    f.write(data_yaml)

print(f"data.yaml сохранён в {dataset_dir}")

Выбрано 6000 изображений с аннотациями
Train: 4200 изображений
Val: 900 изображений
Test: 900 изображений
data.yaml сохранён в /content/drive/MyDrive/CV_SBER/dataset
